In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

/var/folders/xg/5ns375k927gfzbg4c143d4zr0000gn/T/ipykernel_89921/1402149193.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
llm_perf_filtered_df = pd.read_csv('./data/llm_perf_filtered.csv', index_col=0).reset_index(drop=True)

In [3]:
llm_perf_filtered_df.head()

,model,throughput,response_length,latency,energy,gpu,task,parameters_count,energy_per_token,dtype,optimization,quantization,cuda-fp16
0,EleutherAI/gpt-neox-20b,27.4,256,9.34,3013.632,NVIDIA A100-SXM4-80GB,chat,20.0,0.00327,float16,NaN,gptq-4bit+exllama-v1,False
1,EleutherAI/pythia-12b,36.6,256,6.99,2101.248,NVIDIA A100-SXM4-80GB,chat,12.0,0.00228,float16,NaN,gptq-4bit+exllama-v1,False
2,EleutherAI/pythia-12b-deduped,36.6,256,7.00,2082.816,NVIDIA A100-SXM4-80GB,chat,12.0,0.00226,float16,NaN,gptq-4bit+exllama-v1,False
3,EleutherAI/polyglot-ko-12.8b,33.4,256,7.67,2313.216,NVIDIA A100-SXM4-80GB,chat,12.8,0.00251,float16,NaN,gptq-4bit+exllama-v1,False
4,DevaMalla/llama-base-7b,39.1,256,6.54,1815.552,NVIDIA A100-SXM4-80GB,chat,7.0,0.00197,float16,NaN,gptq-4bit+exllama-v1,False


In [4]:
x_range = np.array([
    llm_perf_filtered_df['parameters_count'].min(),
    llm_perf_filtered_df['parameters_count'].max()
]).reshape(-1, 1)

x = llm_perf_filtered_df['parameters_count']
y = llm_perf_filtered_df['energy_per_token']

poly = PolynomialFeatures(degree=1)
X = np.array(x).reshape(-1, 1)
poly.fit(X)
X_poly = poly.transform(X)
x_range_poly = poly.transform(x_range)
model = LinearRegression(fit_intercept=True)
model.fit(X_poly, y)
y_range_poly = model.predict(x_range_poly)

print(f'Coef and intercept for A100: {model.coef_[1]}, {model.intercept_}')

Coef and intercept for A100: 8.910917097396765e-05, 0.0014387038181038398


In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    name=f'Models',
    marker=dict(color='#0b3b36', opacity=0.5)
))

fig.add_trace(go.Scatter(
    x=x_range.squeeze(),
    y=y_range_poly,
    mode='lines',
    name=f'Linear Regression',
    line=dict(color='#00bf63', width=3)
))

fig.update_layout(
    title='Energy/token per model parameters count<br><span style="font-size: 12px">Data from LLM-Perf Leaderboard</span>',
    xaxis=dict(title='#B Parameters'),
    yaxis=dict(title='Energy/token [Wh]', range=[0, 0.01]),
    template='plotly',
    plot_bgcolor='#edf4f3',
    width=800, height=600,
    images=[
        dict(
            source="https://ecologits.ai/assets/logo_light.png",
            xref="paper", yref="paper",
            x=1, y=1,
            sizex=0.28, sizey=0.15,
            xanchor="right", yanchor="bottom"
        )
    ]
)

fig.show()

In [29]:
x_range = np.array([
    llm_perf_filtered_df['parameters_count'].min(),
    llm_perf_filtered_df['parameters_count'].max()
]).reshape(-1, 1)

x = llm_perf_filtered_df['parameters_count']
y = llm_perf_filtered_df['latency'] / llm_perf_filtered_df['response_length']

poly = PolynomialFeatures(degree=1)
X = np.array(x).reshape(-1, 1)
poly.fit(X)
X_poly = poly.transform(X)
x_range_poly = poly.transform(x_range)
model = LinearRegression(fit_intercept=True)
model.fit(X_poly, y)
y_range_poly = model.predict(x_range_poly)

print(f'Coef and intercept for A100: {model.coef_[1]}, {model.intercept_}')

Coef and intercept for A100: 0.0008017124410366496, 0.02234163886672691


In [30]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    name=f'Models',
    marker=dict(color='#0b3b36', opacity=0.5)
))

fig.add_trace(go.Scatter(
    x=x_range.squeeze(),
    y=y_range_poly,
    mode='lines',
    name=f'Linear Regression',
    line=dict(color='#00bf63', width=3)
))

fig.update_layout(
    title='Latency/token per model parameters count<br><span style="font-size: 12px">Data from LLM-Perf Leaderboard</span>',
    xaxis=dict(title='#B Parameters'),
    yaxis=dict(title='Latency/token [s]', range=[0, 0.1]),
    template='plotly',
    plot_bgcolor='#edf4f3',
    width=800, height=600,
    images=[
        dict(
            source="https://ecologits.ai/assets/logo_light.png",
            xref="paper", yref="paper",
            x=1, y=1,
            sizex=0.28, sizey=0.15,
            xanchor="right", yanchor="bottom"
        )
    ]
)

fig.show()
